In [ ]:
import cenpy
import contextily as ctx 
import segregation
%matplotlib inline
import matplotlib as mpl

In [ ]:
import xyzservices.providers as xyz
import matplotlib.pyplot as plt #to make sure there are no errors when plotting a graph
from contextily import add_basemap
import folium 

In [ ]:
import geopandas as gpd 
from census import Census 
from us import states
from shapely.geometry import Polygon

In [ ]:
import pandas as pd

In [ ]:
from contextily import Place


**Census Api**

In [ ]:
acs=cenpy.products.ACS(year="latest")  
acs.filter_tables('POPULATION', by='description')


In [ ]:
c = Census('f5e7cf46eaa83ae8323e6051f74d9ab363d0b957')

In [ ]:
# Sources: https://api.census.gov/data/2019/acs/acs5/variables.html; https://pypi.org/project/census/
bxr_householder_by_race = c.acs5.state_county_tract(fields = ('NAME', 'B25006_001E', # All Race of Householders
                                                                'B25006_003E', # Black Race of Householders
                                                                ), 
                                      state_fips = states.TX.fips, 
                                      county_fips = "029",
                                      tract = "*",
                                      year = 2019)

In [ ]:
bxr_hh_race_df = pd.DataFrame(bxr_householder_by_race)

In [ ]:
print(bxr_hh_race_df.head(2))
print('Shape: ', bxr_hh_race_df.shape)

access shapefile of bexar county. this will coordinate data and alignment with Mercator projection

In [ ]:
bxr_census_tract = gpd.read_file("https://www2.census.gov/geo/tiger/TIGER2019/TRACT/tl_2019_48_tract.zip")


In [ ]:
print(bxr_census_tract.head(2))

print('Shape: ', bxr_census_tract.shape)

# Check shapefile projection
print("\nThe shapefile projection is: {}".format(bxr_census_tract.crs))

In [ ]:
bxr_census_tract.plot('TRACTCE', legend=False, figsize=(10,10))

plt.style.use('dark_background')



performing dataframe operations= https://pygis.io/docs/e_vector_merge_dissolve.html

In [ ]:
# Combine state, county, and tract columns together to create a new string and assign to new column
bxr_hh_race_df["GEOID"] = bxr_hh_race_df["state"] + bxr_hh_race_df["county"] + bxr_hh_race_df["tract"]

In [ ]:
bxr_hh_race_df.head(2)

In [ ]:
# Remove columns
bxr_hh_race_df = bxr_hh_race_df.drop(columns = ["state", "county", "tract"])

# Show updated dataframe
bxr_hh_race_df.head(2)

check column data types 

In [ ]:
# Check column data types for census data
print("Column data types for census data:\n{}".format(bxr_hh_race_df.dtypes))

# Check column data types for census shapefile
print("\nColumn data types for census shapefile:\n{}".format(bxr_census_tract.dtypes))

# Source: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dtypes.html

merge dataframes

In [ ]:
# Join the attributes of the dataframes together
# Source: https://geopandas.org/docs/user_guide/mergingdata.html
bxr_merge = bxr_census_tract.merge(bxr_hh_race_df, on = "GEOID")

# Show result
print(bxr_merge.head(2))
print('Shape: ', bxr_merge.shape)

Subset dataframe. working with bxr_merge shapefile

In [ ]:
# Create new dataframe from select columns
bxr_hh_race_tract = bxr_merge[["STATEFP", "COUNTYFP", "TRACTCE", "GEOID", "geometry", "B25006_001E", "B25006_003E"]]

# Show dataframe
print(bxr_hh_race_tract.head(2))
print('Shape: ', bxr_hh_race_tract.shape)

In [ ]:
# Dissolve and group the census tracts within each county and aggregate all the values together
# Source: https://geopandas.org/docs/user_guide/aggregation_with_dissolve.html
bxr_hh_race_county = bxr_hh_race_tract.dissolve(by = 'TRACTCE', aggfunc = 'mean')

# Show dataframe
print(bxr_hh_race_county.head())
print('Shape: ', bxr_hh_race_county.shape)

In [ ]:
# Get householder by race rate and store values in new column
bxr_hh_race_county["Mean"] = (bxr_hh_race_county["B25006_003E"]/bxr_hh_race_county["B25006_001E"])*100


# Show dataframe
bxr_hh_race_county.head(10)  

In [ ]:
xyz

In [ ]:
# Create subplots
fig, ax =  plt.subplots(figsize = (10,10), dpi=100)
plt.subplots_adjust(bottom=1, right=1, top=2)

# Plot data
# Source: https://geopandas.readthedocs.io/en/latest/docs/user_guide/mapping.html

bxr_plot = bxr_hh_race_county.plot(column= 'Mean', 
                       ax = ax,
                       cmap = 'RdPu',
                       legend = True,
                       aspect='auto')
                                            

#ctx.Place("San Antonio, Tx")

ctx.add_basemap(ax, url=xyz.MapTiler.Basic4326, alpha=.75, zoom=5)
#ctx.add_basemap(ax, crs= 4269, url='http://maps.stamen.com/m2i/#terrain/1000:1000/10/29.4297/-98.5266', alpha=.50, zoom=18)
#ctx.add_basemap(ax, crs=4269, source=ctx.providers.Stamen.TonerLabels, alpha=.90, zoom='auto', attribution_size=12)
#ax.set_axis_off()

# Stylize plots
#plt.style.use('dark_background')


#ax.grid(visible=None) #remove grid lines 

# Set title
ax.set_title('The average total of Black/African American Householders in Bexar County from 2015-2019', fontdict = {'fontsize': '12', 'fontweight' : '3'})

ax.tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)